**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json,Model
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Input

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> During the training phase, the agent needs to explore, or else it would simply exploit its current strategy and that would prevent it from learning. This is a way to force it to explore.

> During the test phase however, there is no need to do so, since the agent doesn't aim at learning better strategies, it simply exploits its policy to maximize its reward. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The ```position``` array encodes the position of the agent on the grid: there is a $1$ where the agent is, and 0 elsewhere (and -1 values on the sides of the grid to prevent the agent from "falling" from it).

> The ```board``` array encodes the maze with its rewards and penalties. Everytime the agent gets somewhere, its value becomes 0, meaning that there is no poison nor cheese left.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        ##### END FILL
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [12]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random10.mp4'))

Win/lose count 12.5/15.0. Average score (-2.5)
Win/lose count 10.0/17.0. Average score (-4.75)
Win/lose count 8.5/12.0. Average score (-4.333333333333333)
Win/lose count 16.0/15.0. Average score (-3.0)
Win/lose count 11.0/18.0. Average score (-3.8)
Win/lose count 8.5/8.0. Average score (-3.0833333333333335)
Win/lose count 7.0/9.0. Average score (-2.9285714285714284)
Win/lose count 12.5/12.0. Average score (-2.5)
Win/lose count 11.0/15.0. Average score (-2.6666666666666665)
Win/lose count 12.5/16.0. Average score (-2.75)
Win/lose count 5.0/10.0. Average score (-2.9545454545454546)
Win/lose count 11.0/16.0. Average score (-3.125)
Win/lose count 11.5/14.0. Average score (-3.076923076923077)
Win/lose count 12.0/13.0. Average score (-2.9285714285714284)
Win/lose count 8.0/12.0. Average score (-3.0)
Win/lose count 11.0/16.0. Average score (-3.125)
Win/lose count 13.0/18.0. Average score (-3.235294117647059)
Win/lose count 16.0/22.0. Average score (-3.388888888888889)
Win/lose count 6.5/11.0.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [13]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        if len(self.memory) > 0:
            return self.memory[np.random.randint(len(self.memory))]
        else:
            RuntimeError('Error: empty memory')

***
The pipeline we will use for training is given below:

In [14]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [33]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #pass
        tmp = self.model.predict(s.reshape(-1,s.shape[0],s.shape[1],s.shape[2]))
        return np.argmax(tmp)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            [sb, n_sb, ab, rb, game_overb] = self.memory.random_access()
            
            #print(self.model.summary())
            #print(self.model.input_shape)
            
            if game_overb:
                tmp = rb 
            else:
                #print(sb.shape)
                input_states[i] = sb
                #print(np.transpose(n_sb).reshape((1,-1)))
                #print(n_sb)
                #print(n_sb.reshape(1,n_sb.shape[0],n_sb.shape[1],n_sb.shape[2]))
                n_preds = self.model.predict(n_sb.reshape(1,n_sb.shape[0],n_sb.shape[1],n_sb.shape[2]))
                tmp = rb + self.discount * np.max(n_preds)
            
            target_q[i] = self.model.predict(sb.reshape(1,n_sb.shape[0],n_sb.shape[1],n_sb.shape[2]))
            target_q[i][ab] = tmp
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states.reshape(-1,n_sb.shape[0],n_sb.shape[1],n_sb.shape[2]), target_q.reshape(-1,1,1,4))


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        
        #Input(shape=(5,5,2))
        model = Sequential([
            #Flatten(),
            Conv2D(50, (5, 5), activation='relu', input_shape=(5,5,3)),
            #Dense(50, activation='relu'),
            Dense(50, activation='relu'),
            Dense(4, activation='relu'),
        ])
        
        ####### END
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [17]:
epochs_train=100 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
print(agent.model.summary())
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1, 1, 50)          2550      
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1, 50)          2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1, 4)           204       
Total params: 5,304
Trainable params: 5,304
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/100 | Loss 0.0109 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/100 | Loss 0.0035 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/100 | Loss 0.0050 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/100 | Loss 0.0092 | Win/lose count 1.5/1.0 (0.5)
Epoch 004/100 | Loss 0.0237 | Win/lose count 1.5/6.0 (-4.5)
Epoch 005/100 | Loss 0.0036 | Win/lose count 3.0/2.0 (1.0)
Epoch 006/100 | Loss 0.0269 | Win/lose count

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [45]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential()
        #print(model.input_shape)
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(5,5,3)))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(Dense(4, activation='relu'))

        ####### END
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [25]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 000/100 | Loss 0.0005 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/100 | Loss 0.0067 | Win/lose count 3.5/7.0 (-3.5)
Epoch 002/100 | Loss 0.0025 | Win/lose count 8.5/5.0 (3.5)
Epoch 003/100 | Loss 0.0280 | Win/lose count 9.5/5.0 (4.5)
Epoch 004/100 | Loss 0.0253 | Win/lose count 7.5/5.0 (2.5)
Epoch 005/100 | Loss 0.0460 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/100 | Loss 0.0023 | Win/lose count 10.5/1.0 (9.5)
Epoch 007/100 | Loss 0.0052 | Win/lose count 8.0/6.0 (2.0)
Epoch 008/100 | Loss 0.0034 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/100 | Loss 0.0016 | Win/lose count 7.5/3.0 (4.5)
Epoch 010/100 | Loss 0.0034 | Win/lose count 4.0/5.0 (-1.0)
Epoch 011/100 | Loss 0.0014 | Win/lose count 10.5/2.0 (8.5)
Epoch 012/100 | Loss 0.0037 | Win/lose count 6.5/2.0 (4.5)
Epoch 013/100 | Loss 0.0474 | Win/lose count 8.5/3.0 (5.5)
Epoch 014/100 | Loss 0.0016 | Win/lose count 8.5/4.0 (4.5)
Epoch 015/100 | Loss 0.0427 | Win/lose count 6.5/2.0 (4.5)
Epoch 016/100 | Loss 0.0086 | Win/lose count 8.0/3.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [28]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 6.5/5.0. Average score (0.5)
Win/lose count 8.0/2.0. Average score (2.3333333333333335)
Win/lose count 3.5/0. Average score (2.625)
Win/lose count 4.5/0. Average score (3.0)
Win/lose count 7.0/0. Average score (3.6666666666666665)
Win/lose count 5.0/0. Average score (3.857142857142857)
Win/lose count 4.0/0. Average score (3.875)
Win/lose count 7.5/3.0. Average score (3.9444444444444446)
Win/lose count 5.0/1.0. Average score (3.95)
Win/lose count 12.0/3.0. Average score (4.409090909090909)
Win/lose count 7.5/3.0. Average score (4.416666666666667)
Win/lose count 7.0/1.0. Average score (4.538461538461538)
Win/lose count 7.5/4.0. Average score (4.464285714285714)
Win/lose count 3.5/3.0. Average score (4.2)
Win/lose count 8.0/4.0. Average score (4.1875)
Win/lose count 4.5/4.0. Average score (3.9705882352941178)
Win/lose count 4.5/3.0. Average score (3.8333333333333335)
Win/lose count 3.5/1.0. Average score (3.763157

In [29]:
HTML(display_videos('cnn_test10.mp4'))

In [30]:
HTML(display_videos('fc_test10.mp4'))

> The agent gets pretty good at avoiding poison and grabing any cheese it sees, yet it tends to quickly get stuck in a place from where it can't see cheese. The exploration is not sufficient, it could earn a lot more if it explored more states. This effect is even worse for low temperature values, because there is less chesse to spot, so the agent gets stuck earlier. At the opposite, for high temperature values the performances improve, because it takes more time to run out of "spottable" cheese. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [46]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            #state, reward, game_over = env.act(action)
            state, reward, game_over = env.act(action, train=True)
            #print(state.shape)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        agent.set_epsilon(1 - e/epoch)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
          
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y] 
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
# ## use those samples of code:
# #In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [47]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
#env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
#agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.1495 | Win/lose count 5.0/104.89999999999999 (-99.89999999999999)
Epoch 001/100 | Loss 0.0275 | Win/lose count 9.0/54.2 (-45.2)
Epoch 002/100 | Loss 0.0865 | Win/lose count 7.5/65.20000000000002 (-57.70000000000002)
Epoch 003/100 | Loss 0.1311 | Win/lose count 11.0/43.20000000000001 (-32.20000000000001)
Epoch 004/100 | Loss 0.0896 | Win/lose count 5.5/63.30000000000002 (-57.80000000000002)
Epoch 005/100 | Loss 0.0749 | Win/lose count 10.5/43.70000000000005 (-33.20000000000005)
Epoch 006/100 | Loss 0.0390 | Win/lose count 7.0/60.700000000000024 (-53.700000000000024)
Epoch 007/100 | Loss 0.0747 | Win/lose count 10.0/49.80000000000002 (-39.80000000000002)
Epoch 008/100 | Loss 0.0369 | Win/lose count 6.5/59.20000000000001 (-52.70000000000001)
Epoch 009/100 | Loss 0.0559 | Win/lose count 7.5/38.000000000000036 (-30.500000000000036)
Epoch 010/100 | Loss 0.0505 | Win/lose count 10.0/53.70000000000003 (-43.70000000000003)
Epoch 011/100 | Loss 0.0446 | Win/lose count 13.0

Epoch 093/100 | Loss 0.0213 | Win/lose count 20.0/22.90000000000002 (-2.90000000000002)
Epoch 094/100 | Loss 0.0575 | Win/lose count 18.0/12.199999999999982 (5.8000000000000185)
Epoch 095/100 | Loss 0.0320 | Win/lose count 27.0/12.599999999999989 (14.400000000000011)
Epoch 096/100 | Loss 0.0381 | Win/lose count 20.5/21.499999999999993 (-0.9999999999999929)
Epoch 097/100 | Loss 0.1300 | Win/lose count 22.0/92.80000000000001 (-70.80000000000001)
Epoch 098/100 | Loss 0.0819 | Win/lose count 25.0/11.799999999999983 (13.200000000000017)
Epoch 099/100 | Loss 0.1239 | Win/lose count 23.0/12.099999999999982 (10.900000000000018)


In [48]:
HTML(display_videos('cnn_train_explore90.mp4'))

In [49]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 13.0/1.0. Average score (12.0)
Win/lose count 25.0/5.0. Average score (16.0)
Win/lose count 21.5/6.0. Average score (15.833333333333334)
Win/lose count 21.5/0. Average score (17.25)
Win/lose count 27.5/5.0. Average score (18.3)
Win/lose count 23.5/9.0. Average score (17.666666666666668)
Win/lose count 25.5/3.0. Average score (18.357142857142858)
Win/lose count 28.0/4.0. Average score (19.0625)
Win/lose count 27.0/6.0. Average score (19.27777777777778)
Win/lose count 24.5/6.0. Average score (19.2)
Win/lose count 22.0/1.0. Average score (19.363636363636363)
Win/lose count 24.0/5.0. Average score (19.333333333333332)
Win/lose count 23.5/2.0. Average score (19.5)
Win/lose count 23.5/6.0. Average score (19.357142857142858)
Win/lose count 23.5/5.0. Average score (19.3)
Win/lose count 24.5/2.0. Average score (19.5)
Win/lose count 26.5/3.0. Average score (19.735294117647058)
Win/lose count 27.5/3.0. Average score (20.0)
Win/lose count 25.5/0. Average score (20.289473684210527)
W

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***